In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


In [3]:
# path = 'output.txt'
# f = open(path, 'a')
# # f.write('Hello World\n')
# time.sleep(3)


In [4]:
# timestamp = datetime.datetime.now().strftime("[%Y-%m-%d %H:%M:%S] ")
# f.write(timestamp+'Hello World\n')

In [5]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [6]:
data_dir = "E:/casper/raw_data_training"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

In [7]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(class_names)
print(device)

['IgA', 'MN']
cuda:0


In [8]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        # timestamp = datetime.datetime.now().strftime("[%Y-%m-%d %H:%M:%S] ")
        # f.write(timestamp+'\n')
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # f.write('Epoch {}/{}'.format(epoch, num_epochs - 1))
        # f.write('\n')
        # f.write('-' * 10)
        # f.write('\n')
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # f.write('{} Loss: {:.4f} Acc: {:.4f}'.format(
            #     phase, epoch_loss, epoch_acc))
            # f.write('\n')
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
        # f.write('\n')
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
#### Finetuning the convnet ####
# Load a pretrained model and reset final fully connected layer.

# from efficientnet_pytorch import EfficientNet
# from torch import nn
# model = EfficientNet.from_pretrained('EfficientNet_V2_S_Weights')
model = models.efficientnet_v2_s()
print(len(dir(model)))
# num_ftrs = model.fc.in_features
num_ftrs = model.classifier[1].in_features

112


In [11]:
model.classifier[1] = nn.Linear(num_ftrs, 2)

In [12]:
model = model.to(device)

In [13]:
criterion = nn.CrossEntropyLoss()

In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [15]:
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [16]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=2)

Epoch 0/1
----------
train Loss: 0.6439 Acc: 0.6315
val Loss: 0.6907 Acc: 0.5971

Epoch 1/1
----------
train Loss: 0.6164 Acc: 0.6569
val Loss: 0.6922 Acc: 0.5569

Training complete in 6m 26s
Best val Acc: 0.597080


In [16]:
model = models.efficientnet_v2_s(weights = True)
print(len(dir(model)))
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=2)

C:\Users\VLSI\anaconda3\envs\casper_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


112
Epoch 0/1
----------
train Loss: 0.5971 Acc: 0.6716
val Loss: 0.7378 Acc: 0.5540

Epoch 1/1
----------
train Loss: 0.5676 Acc: 0.6978
val Loss: 0.6454 Acc: 0.6445

Training complete in 5m 11s
Best val Acc: 0.644526


In [17]:
#### ConvNet as fixed feature extractor ####
# Here, we need to freeze all the network except the final layer.
# We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward()
model_conv = models.efficientnet_v2_s(weights = True)
for param in model_conv.parameters():
    # print(param)
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default

num_ftrs = model_conv.classifier[1].in_features
model_conv.classifier[1] = nn.Linear(num_ftrs, 2)
model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.classifier[1].parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=2)

Epoch 0/1
----------
train Loss: 0.6138 Acc: 0.6596
val Loss: 0.7523 Acc: 0.5650

Epoch 1/1
----------
train Loss: 0.6052 Acc: 0.6711
val Loss: 345.0967 Acc: 0.5839

Training complete in 5m 8s
Best val Acc: 0.583942


In [18]:
model_v2 = models.efficientnet_v2_s(weights = True)
num_ftrs = model_v2.classifier[1].in_features
model_v2.classifier[1] = nn.Linear(num_ftrs, 2)
model_v2 = model_v2.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_v2.parameters(), lr=0.001)
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model_v2 = train_model(model_v2, criterion, optimizer, step_lr_scheduler, num_epochs=20)


Epoch 0/19
----------
train Loss: 0.6063 Acc: 0.6632
val Loss: 0.6708 Acc: 0.6277

Epoch 1/19
----------
train Loss: 0.5615 Acc: 0.7020
val Loss: 0.6289 Acc: 0.6854

Epoch 2/19
----------
train Loss: 0.5502 Acc: 0.7144
val Loss: 0.6031 Acc: 0.7058

Epoch 3/19
----------
train Loss: 0.5390 Acc: 0.7245
val Loss: 0.5609 Acc: 0.7292

Epoch 4/19
----------
train Loss: 0.5196 Acc: 0.7410
val Loss: 0.5471 Acc: 0.7431

Epoch 5/19
----------
train Loss: 0.5123 Acc: 0.7431
val Loss: 0.5257 Acc: 0.7591

Epoch 6/19
----------
train Loss: 0.5080 Acc: 0.7530
val Loss: 0.5610 Acc: 0.7292

Epoch 7/19
----------
train Loss: 0.4991 Acc: 0.7549
val Loss: 0.5373 Acc: 0.7511

Epoch 8/19
----------
train Loss: 0.4959 Acc: 0.7553
val Loss: 0.5238 Acc: 0.7547

Epoch 9/19
----------
train Loss: 0.4904 Acc: 0.7688
val Loss: 0.5699 Acc: 0.7285

Epoch 10/19
----------
train Loss: 0.4985 Acc: 0.7563
val Loss: 0.5196 Acc: 0.7555

Epoch 11/19
----------
train Loss: 0.4921 Acc: 0.7615
val Loss: 0.5374 Acc: 0.7504

Ep

In [19]:
#### ConvNet as fixed feature extractor ####
# Here, we need to freeze all the network except the final layer.
# We need to set requires_grad == False to freeze the parameters so that the gradients are not computed in backward()
model_conv = models.efficientnet_v2_s(weights = True)
for param in model_conv.parameters():
    # print(param)
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default

num_ftrs = model_conv.classifier[1].in_features
model_conv.classifier[1] = nn.Linear(num_ftrs, 2)
model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.classifier[1].parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.6082 Acc: 0.6666
val Loss: 13538.6251 Acc: 0.5496

Epoch 1/19
----------
train Loss: 0.6059 Acc: 0.6675
val Loss: 3807.3322 Acc: 0.5518

Epoch 2/19
----------
train Loss: 0.5986 Acc: 0.6784
val Loss: 919.4916 Acc: 0.6088

Epoch 3/19
----------
train Loss: 0.6119 Acc: 0.6673
val Loss: 12685.1134 Acc: 0.5460

Epoch 4/19
----------
train Loss: 0.6095 Acc: 0.6719
val Loss: 0.6507 Acc: 0.6401

Epoch 5/19
----------
train Loss: 0.6084 Acc: 0.6749
val Loss: 5710.3567 Acc: 0.5964

Epoch 6/19
----------
train Loss: 0.6137 Acc: 0.6693
val Loss: 843.4593 Acc: 0.5861

Epoch 7/19
----------
train Loss: 0.5966 Acc: 0.6747
val Loss: 2003.4994 Acc: 0.5723

Epoch 8/19
----------
train Loss: 0.5945 Acc: 0.6767
val Loss: 37.2705 Acc: 0.5715

Epoch 9/19
----------
train Loss: 0.5883 Acc: 0.6795
val Loss: 0.6783 Acc: 0.5985

Epoch 10/19
----------
train Loss: 0.5895 Acc: 0.6792
val Loss: 9.5115 Acc: 0.6088

Epoch 11/19
----------
train Loss: 0.5909 Acc: 0.6872
val Loss: 